In [1]:
import pandas as pd

In [2]:
#Get the socioeconomic status per zipcode

In [3]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [9]:
socioeconomic_df.head()

,zip,pct_poverty,pct_black,pct_college_plus,pct_mortgage
0,25245,0.116667,0.000000,0.074148,0.176692
1,25268,0.259336,0.000000,0.249664,0.467532
2,25286,0.284118,0.000000,0.045030,0.548263
3,25303,0.105318,0.092253,0.391691,0.607481
4,25311,0.246128,0.153594,0.264175,0.478395


In [10]:
#Read causal effect file(houston)

In [11]:
houston_tx_treatment_effect = pd.read_csv('houston_tx_climate_causal_effect.csv')

In [12]:
houston_tx_treatment_effect.head()

,Unnamed: 0,zip,1,2,3,4,5,6,elderlyReturns,returnsTotalwSalariesWages,...,returnsEducationCredit,risk_group,propensity_score,price_shift,y_0,y_0_corrected,y_1,y_1_corrected,treatment_effect,treatment_effect_no_corr
0,280,77002,0.192192,0.160661,0.171171,0.114114,0.201201,0.160661,0.108108,0.885886,...,0.063063,True,0.308428,-8.733000,11.489105,11.489105,3.136537,-35.347489,-46.836594,-8.352568
1,281,77003,0.250929,0.193309,0.146840,0.100372,0.210037,0.098513,0.109665,0.892193,...,0.055762,False,0.321954,-3.887645,11.743403,-11.309660,6.257732,6.257732,17.567392,-5.485672
2,282,77004,0.300871,0.194774,0.136184,0.086302,0.167854,0.114014,0.184481,0.857482,...,0.060966,True,0.343064,-1.595067,11.679975,11.679975,7.760785,-19.510698,-31.190673,-3.919190
3,283,77005,0.149162,0.082083,0.072374,0.064431,0.143866,0.488085,0.353045,0.742277,...,0.025596,True,0.346551,-0.111441,10.149339,10.149339,-3.896607,7.025772,-3.123567,-14.045946
4,284,77006,0.151919,0.158966,0.163665,0.116680,0.198121,0.210650,0.180110,0.855912,...,0.047768,True,0.283554,-3.160980,11.284557,11.284557,0.872246,-13.351602,-24.636159,-10.412311


In [13]:
#Merge by zip

In [14]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
houston_tx_treatment_effect['zip'] = houston_tx_treatment_effect['zip'].astype(str).str.zfill(5)

C:\Users\user\AppData\Local\Temp\ipykernel_38156\267431552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)


In [15]:
houston_tx_socioeconomic_treatment_effect = pd.merge(socioeconomic_df, houston_tx_treatment_effect, on="zip")

In [16]:
#OLS regression treatment effect ~ socioeconomic variables

In [17]:
import statsmodels.api as sm

X = houston_tx_socioeconomic_treatment_effect[['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']]
X = sm.add_constant(X)
y = houston_tx_socioeconomic_treatment_effect['treatment_effect_no_corr']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     treatment_effect_no_corr   R-squared:                       0.860
Model:                                  OLS   Adj. R-squared:                  0.857
Method:                       Least Squares   F-statistic:                     319.6
Date:                      Thu, 08 May 2025   Prob (F-statistic):           1.36e-87
Time:                              20:39:19   Log-Likelihood:                -396.94
No. Observations:                       213   AIC:                             803.9
Df Residuals:                           208   BIC:                             820.7
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [18]:
# Mean treatment effect
houston_tx_socioeconomic_treatment_effect['treatment_effect_no_corr'].mean()

-2.943236416870285

In [19]:
# Mean treatment effect: Lower 25% & Upper 25%

In [20]:
low_poverty = houston_tx_socioeconomic_treatment_effect[houston_tx_socioeconomic_treatment_effect['pct_poverty'] <= houston_tx_socioeconomic_treatment_effect['pct_poverty'].quantile(0.25)]

In [21]:
low_poverty['treatment_effect_no_corr'].mean()

-7.460176816808292

In [22]:
high_poverty = houston_tx_socioeconomic_treatment_effect[houston_tx_socioeconomic_treatment_effect['pct_poverty'] >= houston_tx_socioeconomic_treatment_effect['pct_poverty'].quantile(0.75)]

In [23]:
high_poverty['treatment_effect_no_corr'].mean()

1.3767354782589951

In [24]:
low_black = houston_tx_socioeconomic_treatment_effect[houston_tx_socioeconomic_treatment_effect['pct_black'] <= houston_tx_socioeconomic_treatment_effect['pct_black'].quantile(0.25)]

In [25]:
low_black['treatment_effect_no_corr'].mean()

-4.719913247219245

In [26]:
high_black = houston_tx_socioeconomic_treatment_effect[houston_tx_socioeconomic_treatment_effect['pct_black'] >= houston_tx_socioeconomic_treatment_effect['pct_black'].quantile(0.75)]

In [27]:
high_black['treatment_effect_no_corr'].mean()

-0.10827415414954339

In [28]:
low_college = houston_tx_socioeconomic_treatment_effect[houston_tx_socioeconomic_treatment_effect['pct_college_plus'] <= houston_tx_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.25)]

In [29]:
low_college['treatment_effect_no_corr'].mean()

0.935256512461053

In [30]:
high_college = houston_tx_socioeconomic_treatment_effect[houston_tx_socioeconomic_treatment_effect['pct_college_plus'] >= houston_tx_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.75)]

In [31]:
high_college['treatment_effect_no_corr'].mean()

-7.6953435685726825

In [32]:
#Read causal effect file(miami)

In [33]:
miami_fl_treatment_effect = pd.read_csv('miami_fl_climate_causal_effect.csv')

In [34]:
miami_fl_treatment_effect.head()

,Unnamed: 0,zip,1,2,3,4,5,6,elderlyReturns,returnsTotalwSalariesWages,...,returnsEducationCredit,risk_group,propensity_score,price_shift,y_0,y_0_corrected,y_1,y_1_corrected,treatment_effect,treatment_effect_no_corr
0,269,33004,0.360234,0.260819,0.147368,0.086550,0.105263,0.039766,0.256140,0.770760,...,0.045614,False,0.442626,16.285307,21.387355,12.233626,22.418959,22.418959,10.185332,1.031603
1,270,33009,0.406321,0.247856,0.130926,0.070880,0.093002,0.051016,0.283070,0.654628,...,0.039729,True,0.489598,1.118005,23.364697,23.364697,25.084658,-23.867029,-47.231725,1.719961
2,271,33010,0.511454,0.307930,0.107048,0.037445,0.030837,0.005286,0.176652,0.730837,...,0.057709,False,0.453532,26.599831,29.512883,24.182191,34.015797,34.015797,9.833607,4.502914
3,272,33012,0.452000,0.313867,0.125867,0.052000,0.047733,0.008533,0.188000,0.737867,...,0.063733,True,0.437269,20.534687,26.535810,26.535810,29.530637,8.957595,-17.578215,2.994827
4,273,33013,0.475083,0.309524,0.115725,0.050941,0.042082,0.006645,0.177187,0.720377,...,0.054817,True,0.446341,25.291373,27.727552,27.727552,31.193661,17.969953,-9.757599,3.466109


In [35]:
#Merge by zip

In [36]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
miami_fl_treatment_effect['zip'] = miami_fl_treatment_effect['zip'].astype(str).str.zfill(5)

C:\Users\user\AppData\Local\Temp\ipykernel_38156\2001426742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)


In [37]:
miami_fl_socioeconomic_treatment_effect = pd.merge(socioeconomic_df, miami_fl_treatment_effect, on="zip")

In [38]:
#OLS regression treatment effect ~ socioeconomic variables

In [39]:
import statsmodels.api as sm

X = miami_fl_socioeconomic_treatment_effect[['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']]
X = sm.add_constant(X)
y = miami_fl_socioeconomic_treatment_effect['treatment_effect_no_corr']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     treatment_effect_no_corr   R-squared:                       0.892
Model:                                  OLS   Adj. R-squared:                  0.889
Method:                       Least Squares   F-statistic:                     355.8
Date:                      Thu, 08 May 2025   Prob (F-statistic):           2.60e-82
Time:                              20:39:19   Log-Likelihood:                -202.02
No. Observations:                       178   AIC:                             414.0
Df Residuals:                           173   BIC:                             429.9
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [40]:
miami_fl_socioeconomic_treatment_effect['treatment_effect_no_corr'].mean()

-0.0004294863123032639

In [41]:
low_poverty = miami_fl_socioeconomic_treatment_effect[miami_fl_socioeconomic_treatment_effect['pct_poverty'] <= miami_fl_socioeconomic_treatment_effect['pct_poverty'].quantile(0.25)]

In [42]:
low_poverty['treatment_effect_no_corr'].mean()

-2.2691349154557505

In [43]:
high_poverty = miami_fl_socioeconomic_treatment_effect[miami_fl_socioeconomic_treatment_effect['pct_poverty'] >= miami_fl_socioeconomic_treatment_effect['pct_poverty'].quantile(0.75)]

In [44]:
high_poverty['treatment_effect_no_corr'].mean()

2.7180483772535555

In [45]:
low_college = miami_fl_socioeconomic_treatment_effect[miami_fl_socioeconomic_treatment_effect['pct_college_plus'] <= miami_fl_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.25)]

In [46]:
low_college['treatment_effect_no_corr'].mean()

2.8603639821809312

In [47]:
high_college = miami_fl_socioeconomic_treatment_effect[miami_fl_socioeconomic_treatment_effect['pct_college_plus'] >= miami_fl_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.75)]

In [48]:
high_college['treatment_effect_no_corr'].mean()

-2.6237558426602123